# PornHub
First a scraped the all the time most viewed videos per country

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

pages_to_scrape = 5
time_range = "a" # all time, other options are last week and last month
ordered_by = "mv" # most viewed

def download_titles(pornhub_url):
    page = requests.get(pornhub_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    videos = soup.find("ul",class_="nf-videos videos search-video-thumbs",id="videoCategory")

    titles = []
    for video_link in videos.find_all("a",class_=""):
        if video_link["href"].startswith("/view_video.php?viewkey="):
            titles.append(video_link.string.strip())
    return titles

def download_country_titles(country_name,country_code):
    titles = []
    for page_i in range(1,pages_to_scrape+1):
        url = f"https://www.pornhub.com/video?o={ordered_by}&t={time_range}&cc={country_code.lower()}&page={page_i}"
        titles += download_titles(url)

    file_name = "video_titles_per_country/" + country_code.upper() + ".csv"
    pd.DataFrame({"Titles":titles}).to_csv(file_name,index=False)

countries = pd.read_csv("countries.csv")
countries.head()

,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Andorra,AD,AND,20
1,United Arab Emirates (the),AE,ARE,784
2,Afghanistan,AF,AFG,4
3,Antigua and Barbuda,AG,ATG,28
4,Anguilla,AI,AIA,660


In [ ]:
for _, country in countries.iterrows():
    try:
        country_code = country["Alpha-2 code"]
        country_name = country["Country"]
        download_country_titles(country_name,country_code)
        print("Sucessfully processed",country_name)
    except Exception as e:
        print("Problem processing", country_name)
        print(e)
    time.sleep(1) # Requesting too fast might produce pornhub banning our IP

Some countries doesn't have a custom videos list and they just share the same. So I discard those.

In [ ]:
import os

def cleaning_based_on_country(country_code):
    csvs = "video_titles_per_country/"
    all_reapeated_csv = csvs + country_code + ".csv"
    all_repeated_csv = pd.read_csv(all_reapeated_csv).values.tolist()
    for file in os.listdir(csvs):
        if file.endswith(".csv"):
            if pd.read_csv(csvs + file).values.tolist() == all_repeated_csv:
                os.rename(csvs + file, "video_titles_not_available/" + file)

cleaning_based_on_country("SM")
cleaning_based_on_country("AQ")

## Extracting Important Keywords from Text with TF-IDF and Python's Scikit-Learn 

I used [this](https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/) as the base for the keywords extraction.

In [59]:
import re, os
from langdetect import detect
from googletrans import Translator
import pandas as pd

# Some titles are in other languages than english so I translate them
def normalize_language(text, target_language="en"):
    try:
        if len(text) > 3 and detect(text) != target_language:
            translator = Translator()
            return translator.translate(text, dest=target_language).text
    except Exception as e:
        pass
    return text

def pre_process(text):
    text=text.lower()
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)

    text=normalize_language(text)
    
    return text

def pre_process_lines(texts):
    return "\n".join(map(pre_process,texts))

normalize_language("hola como estas?")

"Hello what's up?"

In [3]:
from tqdm import tqdm

documents = []
files = sorted(filter(lambda file: file.endswith(".csv"),os.listdir("video_titles_per_country/")))
for file in tqdm(files):
    with open("video_titles_per_country/"+file,'r') as f:
        documents.append(pre_process_lines(f.readlines()))

df_idf = pd.DataFrame({"text":documents,"files":files})

## Creating the IDF

### CountVectorizer to create a vocabulary and generate word counts
The next step is to start the counting process. We can use the CountVectorizer to create a vocabulary from all the text in our `df_idf['text']` and generate counts for each row in `df_idf['text']`. The result of the last two lines is a sparse matrix representation of the counts, meaning each column represents a word in the vocabulary and each row represents the document in our dataset where the values are the word counts. Note that with this representation, counts of some words could be 0 if the word did not appear in the corresponding document.

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

#get the text column 
docs=df_idf['text'].tolist()
cv=CountVectorizer(max_df=0.85)
word_count_vector=cv.fit_transform(docs)

In [63]:
cv=CountVectorizer(max_df=0.85,max_features=10000)
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(41, 2522)

### TfidfTransformer to Compute Inverse Document Frequency (IDF) 
In the code below, we are essentially taking the sparse matrix from CountVectorizer to generate the IDF when you invoke `fit`. An extremely important point to note here is that the IDF should be based on a large corpora and should be representative of texts you would be using to extract keywords. I've seen several articles on the Web that compute the IDF using a handful of documents. To understand why IDF should be based on a fairly large collection, please read this [page from Standford's IR book](https://nlp.stanford.edu/IR-book/html/htmledition/inverse-document-frequency-1.html).

In [66]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Let's look at some of the IDF values:

In [67]:
tfidf_transformer.idf_

array([3.63905733, 4.04452244, 3.63905733, ..., 4.04452244, 4.04452244,
       4.04452244])

## Computing TF-IDF and Extracting Keywords

Once we have our IDF computed, we are now ready to compute TF-IDF and extract the top keywords. In this example, we will extract top keywords for the questions in `data/stackoverflow-test.json`. This data file has 500 questions with fields identical to that of `data/stackoverflow-data-idf.json` as we saw above. We will start by reading our test file, extracting the necessary fields (title and body) and get the texts into a list.

In [68]:
docs_test=df_idf['text'].tolist()

In [69]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

The next step is to compute the tf-idf value for a given document in our test set by invoking `tfidf_transformer.transform(...)`. This generates a vector of tf-idf scores. Next, we sort the words in the vector in descending order of tf-idf values and then iterate over to extract the top-n items with the corresponding feature names, In the example below, we are extracting keywords for the first document in our test set. 

The `sort_coo(...)` method essentially sorts the values in the vector while preserving the column index. Once you have the column index then its really easy to look-up the corresponding word value as you would see in `extract_topn_from_vector(...)` where we do `feature_vals.append(feature_names[idx])`.

In [74]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)


In [71]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_test[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])


Now let's look at keywords generated for a much longer question: 


## Generate keywords for a batch of documents

In [93]:
pd.set_option('display.max_colwidth', -1)
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
n = 15
for i in range(tf_idf_vector.shape[0]):
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())
    keywords=extract_topn_from_vector(feature_names,sorted_items,n)
    results.append(list(keywords.keys()))

df=pd.DataFrame([results[i] for i in range(len(docs))],columns=[f'keyword_{i}' for i in range(n)])
df["country"] = df_idf["files"].apply(lambda file : file[:2])
df = df[["country"]+[f'keyword_{i}' for i in range(n)]]
df

,country,keyword_0,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,keyword_9,keyword_10,keyword_11,keyword_12,keyword_13,keyword_14
0,AR,argentina,jordi,enp,flower,babes,fifa,ending,spanish,happy,seduction,argentinian,asshole,mk,khalifa,whore
1,AT,german,lucy,cat,anal,public,devil,or,tiny,surprise,angel,solo,cosplay,gorgeous,wet,stranger
2,AU,farrah,asian,bestie,abraham,window,girlsoutwest,australian,aussie,time,girlcum,fanboy,catches,realtor,games,bff
3,BE,french,solo,reverse,cosplay,tori,sacha,caprice,belgian,tiny,see,realtor,teenmegaworld,sleep,bounce,anya
4,BG,bg,bulgarian,solo,peta,jensen,tiny,cosplay,yako,varti,trial,tames,sweetly,suzi,stubborn,shake
5,BR,anal,maru,dread,brazilian,vox,skylar,karv,chi,brazil,stella,raee,portuguese,joi,omg,free
6,CA,farrah,asian,tiny,bestie,abraham,biggest,realtor,orgasms,janice,griffith,daughterswap,fps,dared,exxxtrasmall,surprising
7,CH,sloppy,deepthroat,swallow,facial,gagging,blowjob,solo,wet,slap,deepest,head,facialized,glasses,masturbation,cosplay
8,CL,jordi,enp,khalifa,ending,teencurves,latina,gameplay,fifa,happy,chilean,stella,bangbros,booty,raee,mk
9,CZ,publicagent,czech,public,agent,car,glory,hole,paid,swap,bridge,cash,solo,ever,cosplay,wanker


In [94]:
df.to_csv("pornhub_keywords_per_country.csv",index=False)